[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Image Processing with Python

## Interpolation by the DFT

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 23/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import itertools
import math
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, Literal, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants



In [ ]:
# Fixel Algorithms Packages


In [ ]:
# General Auxiliary Functions

def IsOdd( inNum: int ) -> bool:
    """
    Check whether an integer is odd.

    Parameters
    ----------
    inNum : int
        The input integer to check.

    Returns
    -------
    bool
        `True` if the input integer is odd, `False` otherwise.

    Notes
    -----
    - The function uses the bitwise AND operation (`& 1`) to determine if the input number is odd.
      An integer is odd if its least significant bit is 1 in binary representation.
    - For example:
        - `5` in binary is `101`, and `5 & 1` equals `1`, so it is odd.
        - `4` in binary is `100`, and `4 & 1` equals `0`, so it is even.

    Examples
    --------
    >>> IsOdd(5)
    True
    >>> IsOdd(4)
    False
    """

    return inNum & 1

def IsEven( inNum: int ) -> bool:
    """
    Check whether an integer is even.

    Parameters
    ----------
    inNum : int
        The input integer to check.

    Returns
    -------
    bool
        `True` if the input integer is even, `False` otherwise.

    Notes
    -----
    - The function uses the bitwise AND operation (`& 1`) to determine if the input number is odd.
      An integer is even if its least significant bit is 0 in binary representation.
    - For example:
        - `5` in binary is `101`, and `5 & 1` equals `1`, so it is odd.
        - `4` in binary is `100`, and `4 & 1` equals `0`, so it is even.

    Examples
    --------
    >>> IsEven(5)
    False
    >>> IsEven(4)
    True
    """

    return inNum & 1

def DftReSample( vX: np.ndarray, outLen: int ) -> np.ndarray:
    """
    Resamples a 1D signal to a specified length using the Discrete Fourier Transform (DFT).

    This function performs resampling (UpSampling or DownSampling) in the frequency domain using
    "Ideal Low Pass" which is equivalent ot the Dirichlet Kernel interpolation ("Sinc Interpolation") on time domain.
    The method ensures energy preservation by Parseval's theorem and maintains conjugate symmetry in the frequency domain.

    Parameters
    ----------
    vX : np.ndarray
        Input 1D signal to be resampled. Must be a one dimensional array.
    outLen : int
        The desired length of the resampled signal.

    Returns
    -------
    np.ndarray
        The resampled signal of length `outLen`.

    Raises
    ------
    ValueError
        If the input `vX` is not a one dimensional array.

    Notes
    -----
    - The function operates in the frequency domain by applying zero padding (UpSampling) 
      or truncation (DownSampling) to the Fourier coefficients. This is equivalent to Sinc 
      Interpolation in the time domain.
    - The function accounts for the symmetry of the DFT coefficients, ensuring proper treatment
      of the Nyquist frequency for real valued signals.

    Algorithm
    ---------
    - If the output length `outLen` is greater than the input length, the signal is up-sampled by
      zero-padding the DFT coefficients symmetrically.
    - If `outLen` is smaller than the input length, the signal is down-sampled by truncating the
      DFT coefficients, which is equivalent to low-pass filtering.
    - In both cases, the resampled signal is normalized by the interpolation factor to preserve energy.

    References
    ----------
    - [The Proper Way to Do Sinc Downsampling (DFT Downsampling) for Uniformly Sampled Discrete Signals 
       with Finite Number of Samples](https://dsp.stackexchange.com/questions/72433)

    Examples
    --------
    >>> import numpy as np
    >>> vX = np.sin(2 * np.pi * np.linspace(0, 1, 100))  # A sine wave
    >>> vY = DftReSample(vX, 200)  #<! UpSample to 200 points
    >>> vZ = DftReSample(vX, 50)   #<! DownSample to 50 points
    """

    if (np.ndim(vX) != 1):
        raise ValueError('The input must be a vector')
    
    inLen = len(vX)
    
    if (outLen == inLen):
        return np.copy(vX)
    
    interpFactor = outLen / inLen
    vXDft = np.fft.fft(vX)

    if (outLen > inLen):
        # UpSample
        halfNSamples = inLen // 2
        if IsOdd(inLen):
            vXDftInt = interpFactor * np.r_[(vXDft[:(halfNSamples + 1)], np.zeros_like(vXDft, shape = outLen - inLen), vXDft[(halfNSamples + 1):])]
        else:
            vXDftInt = interpFactor * np.r_[(vXDft[:halfNSamples], vXDft[halfNSamples] / 2, np.zeros_like(vXDft, shape = outLen - inLen - 1), vXDft[halfNSamples] / 2, vXDft[(halfNSamples + 1):])]
    elif (outLen < inLen):
        # DownSample
        # Equivalent of applying an LPF
        halfNSamples = outLen // 2
        if IsOdd(outLen):
            vXDftInt = interpFactor * np.r_[(vXDft[:halfNSamples], vXDft[(inLen - halfNSamples):])]
        else:
            # If the signal is known to be real the Nyquist Sample should be real.
            # It is not enforced here, should be enforced either specifically or by using `fft(..., 'symmetric');`.
            vXDftInt = interpFactor * np.r_[(vXDft[:halfNSamples], vXDft[halfNSamples] / 2, vXDft[(halfNSamples + 1):])]
    
    vY = np.real(np.fft.ifft(vXDftInt))


    return vY



* <font color='blue'>(**!**)</font> Optimize the code to use `np.fft.rfft()`.

In [ ]:
vX = np.array([1.0, 0.0, -1.0, 0.0, 1.0, 0.0, -1.0, 0.0]) #!< Cosine

vX = np.array([1.0, 0.0, -1.0, 0.0, 1.0, 0.0, -1.0])

vY = DftReSample(vX, 94)
vY.shape

In [ ]:
plt.plot(vY);

## Resampling in Frequency Domain

The ideal sampling and reconstruction using the Fourier Domain is by the [Sinc Kernel](https://en.wikipedia.org/wiki/Sinc_filter) (See [Whittaker Shannon Interpolation Formula](https://en.wikipedia.org/wiki/Whittaker%E2%80%93Shannon_interpolation_formula)).  
The [Sinc Function](https://en.wikipedia.org/wiki/Sinc_function) is the ideal LPF on the _Frequnecy Domain_.

When dealing with **Discrete Signals** the Sinc Kernel is replaced by the [Dirichlet Kernel](https://en.wikipedia.org/wiki/Dirichlet_kernel).

This notebook examines the performance per implementation.

* <font color='brown'>(**#**)</font> See [The Proper Way to Do Sinc Downsampling (DFT Downsampling) for Uniformly Sampled Discrete Signals with Finite Number of Samples](https://dsp.stackexchange.com/questions/72433).

In [ ]:
# Parameters

# imgUrl = 'https://i.imgur.com/3BbIXdH.png' #<! A turtle climbing the Everest!
imgUrl = 'https://i.postimg.cc/63rN33GZ/3BbIXdH.png' #<! A turtle climbing the Everest!

lImageSize      = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
lKernelRadius   = [5, 10, 15, 20, 25, 30, 35, 40]

lImageSize      = [100, 200, 300, 400]
lKernelRadius   = [5, 10, 15]


## Generate Data

This section loads the image used to evaluate the results.  

In [ ]:
# Load / Generate Data

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2) #<! Single channel image

In [ ]:
# Display the Data

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mI, cmap = 'gray')
hA.set_title(f'Input Image');

In [ ]:
# Pandas Data Frame

numComb  = len(lImageSize) * len(lKernelRadius)
dRunTime = {
    'Image Size'                            : [],
    'Kernel Radius'                         : [],
    'Direct Convolution Run Time [Mili Sec]': [],
    'DFT Convolution Run Time [Mili Sec]'   : [],
    }
for (imgSize, krnlRadius) in itertools.product(lImageSize, lKernelRadius):
    dRunTime['Image Size'].append(imgSize)
    dRunTime['Kernel Radius'].append(krnlRadius)

    mII = ski.transform.resize(mI, (imgSize, imgSize), preserve_range = True)
    mKK = np.random.rand(2 * krnlRadius + 1, 2 * krnlRadius + 1)
    mKK /= np.sum(mKK)

    runTime = timeit.timeit('sp.signal.convolve(mII, mKK, method = "direct")', number = 3, setup = 'import scipy as sp', globals = {'mII': mII, 'mKK': mKK})
    dRunTime['Direct Convolution Run Time [Mili Sec]'].append(runTime * 1e3)

    runTime = timeit.timeit('sp.signal.convolve(mII, mKK, method = "fft")', number = 3, setup = 'import scipy as sp', globals = {'mII': mII, 'mKK': mKK})
    dRunTime['DFT Convolution Run Time [Mili Sec]'].append(runTime * 1e3)

dfRunTime = pd.DataFrame(dRunTime)
dfRunTime

* <font color='red'>(**?**)</font> What about the cases for Gaussian Blur and Box Blur?